### Discription:
In this notebook, I test the hypothesis that aggressive feature crossing can lead to overfitting if not applied carefully.

I compare model performance with and without engineered interaction features using a structured preprocessing pipeline and hyperparameter tuning.

The results highlight how excessive manual feature interactions can sometimes hurt generalization, especially when using tree-based boosting models.

Hope others will find this helpful!!

## Importing dataset:

In [19]:
import pandas as pd

train_path = '/kaggle/input/competitions/playground-series-s6e2/train.csv'
train_df = pd.read_csv(train_path)
train_df.head() #checkng if data has loaded safely

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


## Cleaning data:

In [20]:
target = 'Heart Disease'

#Encoding the target
train_df[target] = train_df[target].map({'Absence': 0, 'Presence': 1})

#removing redundant features and seperating features and target:
X = train_df.drop(columns=['id',target], axis=1)
y = train_df[target]

## Feature engineering:

Today I learned that feature crosses can cause overfitting if not done properly, so this iteration will try to improve validation and final test score by not performing feature crossing. 

Though its true that gradientboosting algorithm like the one we'll be using here "XGboost" can learn these interactions themselves, providing these features ready-made could add noise.

In [21]:
#Just for example, feature crosses I tried earlier

def create_features(df):
    #Feature crosses/ Interaction features: (Optinal, some models are able to learn these relatonships on there own)
    df['Y1'] = df['BP'] * df['Cholesterol']
    df['Y2'] = df['Number of vessels fluro'] * df['Slope of ST']
    df['Y3'] = df['Cholesterol'] * df['Slope of ST']
    df['Y4'] = df['Cholesterol'] * df['Number of vessels fluro']
    df['Y5'] = df['BP'] * df['Slope of ST']
    df['Y6'] = df['BP'] * df['Number of vessels fluro']

    return df

In [22]:
# X = create_features(X) #since not using feature crosses

### Custom transformers:


In [23]:

from sklearn.base import BaseEstimator, TransformerMixin

#Binning Transformer
class Binning(BaseEstimator, TransformerMixin):
    def __init__(self, col_to_bin, num_bins, new_col_name ,labels=None):
        self.col_to_bin = col_to_bin
        self.num_bins = num_bins
        self.labels = labels
        self.new_col_name = new_col_name

    def fit(self, X, y=None):
        X = X.copy()
        _, self.bin_edges = pd.cut(X[self.col_to_bin], bins=self.num_bins, labels=False, retbins=True)
        return self

    def transform(self,X):
        X = X.copy() 
        X[self.new_col_name] = pd.cut(X[self.col_to_bin], bins=self.bin_edges, labels=False)
        return X

#GroupMean Transformer
class GroupMeanEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, groupby_col, agg_col, new_col_name):
        self.groupby_col = groupby_col
        self.agg_col = agg_col
        self.new_col_name = new_col_name

    def fit(self,X,y=None):
        self.means = X.groupby(self.groupby_col,observed=True)[self.agg_col].mean()
        return self

    def transform(self,X):
        X = X.copy()
        X[self.new_col_name] = X[self.groupby_col].map(self.means)
        return X

#FrequencyEncoder Transformer
class FreqEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cat_cols, normalize=True):
        self.cat_cols = cat_cols
        self.normalize = normalize
        self.freq_maps = {}

    def fit(self, X, y=None):
        for col in self.cat_cols:
            self.freq_maps[col] = X[col].value_counts(normalize=self.normalize)
        return self

    def transform(self, X):
        X = X.copy()

        for col in self.cat_cols:
            X[col + '_freq'] = X[col].map(self.freq_maps[col])
            X[col + '_freq'] = X[col + '_freq'].fillna(0) 
        return X

### Target encoding transformer:
Target Encoding = replacing a categorical value with the mean of the target for that category. 

This is a new feature engineering technique that I picked up recently as well and is pretty effective. 

In [24]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cat_cols):
        self.cat_cols = cat_cols
        self.mean_maps = {}
        self.global_mean = None
        
    def fit(self, X, y):
        X = X.copy()
        self.global_mean = y.mean()

        for col in self.cat_cols:
            self.mean_maps[col] = y.groupby(X[col]).mean()
        return self

    def transform(self, X):
        X = X.copy()
        
        for col in self.cat_cols:
            X[col + "_TE"] = X[col].map(self.mean_maps[col])
            X[col + "_TE"] = X[col + "_TE"].fillna(self.global_mean) #handling unseen values
        return X

## Building data preprocessing pipelines:

In [25]:
cat_cols = ['Sex','Chest pain type','FBS over 120','Exercise angina','EKG results']

In [26]:
from sklearn.pipeline import Pipeline 

preprocessor = Pipeline([
    ('Binning', Binning(col_to_bin='Age', num_bins=3, new_col_name='Age_bins')),
    ('GroupMeanEncoder_BP', GroupMeanEncoder(groupby_col='Age_bins', agg_col='BP', new_col_name='X1')),
    ('GroupMeanEncoder_Cholesterol', GroupMeanEncoder(groupby_col='Age_bins', agg_col='Cholesterol', new_col_name='X2')),
    ('GroupMeanEncoder_HR', GroupMeanEncoder(groupby_col='Age_bins', agg_col='Max HR', new_col_name='X3')),
    ('FreqEncoding', FreqEncoder(cat_cols=cat_cols)),
    ('TargetEncoding', TargetEncoder(cat_cols=cat_cols))
])

## Optuna Tuning:

### XGBoost:

In [27]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):

    params = {
        # Core learning parameters
        "n_estimators": trial.suggest_int("n_estimators", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        
        # Tree complexity control
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        
        # Sampling 
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1.0),
        
        # Regularization 
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),

        # Performance
        "random_state": 42,
        "eval_metric": "logloss",
        "tree_method": "hist",   
        "verbosity": 0,
        "device":'cuda'
    }

    model = Pipeline([
        ('prep', preprocessor),
        ('XGB', XGBClassifier(**params))
    ])

    score = cross_val_score(model, X, y, cv=5, scoring="roc_auc").mean()

    return score


In [28]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)

[I 2026-02-17 17:36:15,226] A new study created in memory with name: no-name-f580dc15-984d-4223-9aaf-8b8bc9385a20
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  w

In [29]:
# print(study.best_params)
# print(study.best_value)

{'n_estimators': 2634, 'learning_rate': 0.0468112533159457, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 1.4373368015867447, 'subsample': 0.7758150044556165, 'colsample_bytree': 0.39762653718463103, 'reg_alpha': 0.1260400161073486, 'reg_lambda': 0.00014786267797526253}
0.9555482973765608


Best hyperparameter set and score found when using **feature crosses**:
* Hyperparameter set: {'n_estimators': 2997, 'learning_rate': 0.02897722764540242, 'max_depth': 3, 'min_child_weight': 2, 'gamma': 2.214493292110104, 'subsample': 0.7125778371354599, 'colsample_bytree': 0.934351105375458, 'reg_alpha': 2.709037256626792, 'reg_lambda': 0.0019825317162724676}
* Score: 0.9554529240396

Best hyperparameter set and score found when **not** using **feature crosses**:

* Hyperparameter set: {'n_estimators': 2634, 'learning_rate': 0.0468112533159457, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 1.4373368015867447, 'subsample': 0.7758150044556165, 'colsample_bytree': 0.39762653718463103, 'reg_alpha': 0.1260400161073486, 'reg_lambda': 0.00014786267797526253}
* Score: 0.9555482973765608

As can be clearly seen that without feature crosses the model has performed better in CV during Optuna tuning

## Retraining the model:
Using the complete data and learned best parameters to retrain a new model on complete train data

In [30]:
best_xgb_model = Pipeline([
        ('prep', preprocessor),
        ('XGB', XGBClassifier(
            n_estimators=2634,
            learning_rate=0.0468112533159457,
            max_depth=3,
            min_child_weight=5,
            gamma=1.4373368015867447,
            subsample=0.7758150044556165,
            colsample_bytree= 0.39762653718463103,
            reg_alpha=0.1260400161073486,
            reg_lambda=0.00014786267797526253,
            
            random_state=42,
            eval_metric="logloss",
            tree_method="hist",
            device="cuda",
            verbosity=0
        ))
    ])

In [31]:
best_xgb_model.fit(X,y)

Pipeline(steps=[('prep',
                 Pipeline(steps=[('Binning',
                                  Binning(col_to_bin='Age',
                                          new_col_name='Age_bins',
                                          num_bins=3)),
                                 ('GroupMeanEncoder_BP',
                                  GroupMeanEncoder(agg_col='BP',
                                                   groupby_col='Age_bins',
                                                   new_col_name='X1')),
                                 ('GroupMeanEncoder_Cholesterol',
                                  GroupMeanEncoder(agg_col='Cholesterol',
                                                   groupby_col='Age_bins',
                                                   new_col_name='X2')),
                                 ('GroupMeanEncoder_HR',
                                  Grou...
                               gamma=1.4373368015867447, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0468112533159457, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=3,
                               max_leaves=None, min_child_weight=5, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=2634, n_jobs=None,
                               num_parallel_tree=None, ...))])

## Preparing test data:

In [32]:
test_path = '/kaggle/input/competitions/playground-series-s6e2/test.csv'
test_df = pd.read_csv(test_path)
test_df.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium
0,630000,58,1,3,120,288,0,2,145,1,0.8,2,3,3
1,630001,55,0,2,120,209,0,0,172,0,0.0,1,0,3
2,630002,54,1,4,120,268,0,0,150,1,0.0,2,3,7
3,630003,44,0,3,112,177,0,0,168,0,0.9,1,0,3
4,630004,43,1,1,138,267,0,0,163,0,1.8,2,0,7


In [33]:
# Removing redundant features
X_test = test_df.drop('id', axis=1)

#applying the create_features on test set
# X_test = create_features(X_test) #since not using feature crosses

## Predicting on the test set:

In [34]:
y_pred_xgb = best_xgb_model.predict_proba(X_test)[:,1]

/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


## Preparing submission csv:

In [35]:
submission = pd.DataFrame({
    'id': test_df['id'],
    target: y_pred_xgb
})

In [36]:
submission.to_csv('submission.csv', index=False)

On submission the score increased as well from 0.95363 --> 0.95371. Thus this experiment that feature crosses can cause the model to overfit on training data/ adds noise than help it to achive better results on unseen data, if not done properly.

Hope it was helpful!!!!

Plzz do upvote the notebook if you found it helpful.

Thanks for reading this all the way!!!